In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from collections import Counter
from os.path import join as oj
from copy import deepcopy
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('dark_background')
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from viz import *
import validate
import data

# out_dir = 'results/aug7_1' # combined the different abdTender values
out_dir = 'results/aug6_6/' # has a lot of results
# out_dir = 'results/aug7_logistics' # has hyperparams for logistic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models

In [6]:
def load_results(out_dir):
    r = []
    for fname in tqdm(os.listdir(out_dir)):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        d['param_name'] = fname[:-4]
        r.append(pd.Series(d))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('param_name')
    return r

results = load_results(out_dir)
# print(results.keys())

100%|██████████| 203/203 [00:02<00:00, 88.79it/s]


## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [7]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
# r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
# r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)
# r = r[r.model_type == 'grl']
# r = r[r.model_type == 'brl']
r = r[r['sensitivity_cv'] > 0.8] # what fraction of true pos do we return?
r = r[r['specificity_cv'] > 0.4] # what fraction of true negs do we return?

r = r.sort_values(by=['sensitivity_cv', 'specificity_cv'], ascending=False)
r = r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2',
       'fn_cv', 'tn_cv', 'fp_cv', 'tp_cv',
       'precision_cv', 'roc_auc_cv', 
#        'f1_cv', 'balanced_accuracy_cv', 'accuracy_cv',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
# r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r.head(15)

,sensitivity_cv,specificity_cv,sensitivity_test1,specificity_test1,sensitivity_test2,specificity_test2,fn_cv,tn_cv,fp_cv,tp_cv,precision_cv,roc_auc_cv,fn_test1,tn_test1,fp_test1,tp_test1,fn_test2,tn_test2,fp_test2,tp_test2
param_name,,,,,,,,,,,,,,,,,,,,
logistic_select_stab_lasso=6_sample_weights=500,0.969,0.522,0.877,0.513,0.984,0.507,4,4129,3774,126,0.032,0.898,9,2021,1917,64,1,1077,1049,61
logistic_select_stab_lasso=7_sample_weights=750,0.969,0.509,0.890,0.493,0.968,0.496,4,4026,3877,126,0.031,0.896,8,1943,1995,65,2,1055,1071,60
logistic_select_stab_lasso=6_sample_weights=750,0.969,0.504,0.904,0.484,0.984,0.476,4,3983,3920,126,0.031,0.897,7,1905,2033,66,1,1011,1115,61
logistic_select_stab_lasso=6_sample_weights=1000,0.969,0.504,0.904,0.484,0.984,0.476,4,3983,3920,126,0.031,0.897,7,1905,2033,66,1,1011,1115,61
logistic_select_stab_lasso=7_sample_weights=500,0.962,0.546,0.863,0.587,0.952,0.556,5,4312,3591,125,0.034,0.897,10,2312,1626,63,3,1182,944,59
logistic_select_stab_lasso=5_sample_weights=500,0.954,0.531,0.863,0.524,0.984,0.516,6,4193,3710,124,0.032,0.884,10,2063,1875,63,1,1097,1029,61
logistic_select_stab_lasso=10_sample_weights=1000,0.954,0.434,0.904,0.369,0.968,0.319,6,3428,4475,124,0.027,0.888,7,1455,2483,66,2,678,1448,60
logistic_select_stab_lasso=10_sample_weights=750,0.946,0.492,0.904,0.446,0.968,0.393,7,3886,4017,123,0.030,0.890,7,1756,2182,66,2,835,1291,60
logistic_select_stab_lasso=10_sample_weights=500,0.938,0.557,0.863,0.542,0.952,0.492,8,4399,3504,122,0.034,0.893,10,2135,1803,63,3,1046,1080,59


# visualize best model

In [ ]:
# best_model = 'logistic_select_lasso=7_sample_weights=750_h=2_c1=True_c2=True'
# best_model = 'logistic_select_stab_lasso=5_sample_weights=500_h=2_c1=True_c2=True'
# best_model = 'logistic_select_lasso=7_sample_weights=500_h=2_c1=True_c2=True'
# logistic_select_stab_lasso=5_sample_weights=750' #logistic_select_rf=5_smote=5'
r = results[results.index.str.contains(best_model)].iloc[0]

d = pd.DataFrame.from_dict({
    'feats': r['feat_names_selected'],
    'imps': r['imps_best'],
})
d = d.iloc[np.argsort(d['imps'])[::-1]]
d = d.round(2)
d.fillna(0).style.background_gradient(cmap=cm_rev) #, axis=0) #, subset=subset) # rows differently colored

In [5]:
d = d[np.abs(d['imps']) >= 0.05]
d.round(1).fillna(0).style.background_gradient(cmap=cm_rev) #, axis=0) #, subset=subset) # rows differently colored

,feats,imps
2,AbdTrauma_or_SeatBeltSign_yes,0.6
5,DecrBreathSound_yes,0.4
0,AbdDistention_or_AbdomenPain_yes,0.2
6,GCSScore_Full_yes,-1
1,AbdTenderDegree_None,-1.1


In [6]:
d = d[np.abs(d['imps']) >= 0.5]
d.round(0).fillna(0).style.background_gradient(cmap=cm_rev)

,feats,imps
2,AbdTrauma_or_SeatBeltSign_yes,1
6,GCSScore_Full_yes,-1
1,AbdTenderDegree_None,-1


In [7]:
r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2']]

sensitivity_cv       0.969
specificity_cv       0.525
sensitivity_test1    0.877
specificity_test1    0.516
sensitivity_test2    0.984
specificity_test2    0.521
Name: logistic_select_lasso=7_sample_weights=750_h=2_c1=True_c2=True, dtype: object

## re-evaluate the model

In [70]:
normalize = False # were normalized during training, but may not be necessary
feat_names = r.feat_names_selected
outcome_def = 'iai_intervention'  # output
train_idxs = data.pecarn_train_idxs
test_idxs1 = data.pecarn_test_idxs
test_idxs2 = data.psrc_train_idxs + data.psrc_test_idxs

df_pecarn, df_psrc, common_feats, filtered_feats_pecarn, filtered_feats_psrc = data.load_it_all(dummy=True)
df = df_pecarn[common_feats].append(df_psrc[common_feats])


X = df[feat_names]
# X = (X - X.mean()) / X.std() - things were normalize
y = df[outcome_def].values

# split data based on cv_fold
cv_idxs = train_idxs[:r.idx_best] + train_idxs[r.idx_best+1:]
idxs_train = df.cv_fold.isin(train_idxs)

X_train, y_train = X[idxs_train], y[idxs_train]
idxs_test1 = df.cv_fold.isin(test_idxs1)
X_test1, Y_test1 = X[idxs_test1], y[idxs_test1]
idxs_test2 = df.cv_fold.isin(test_idxs2)
X_test2, Y_test2 = X[idxs_test2], y[idxs_test2]
#     print('shapes', X_train.shape[0], X_test1.shape[0], X_test2.shape[0])

/accounts/projects/vision/.local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [71]:
m = deepcopy(r['model_best'])
# m.coef_
m.coef_ = m.coef_.round(0)

In [72]:
for name, x, y, in zip(['cv', 'test1', 'test2'],
                       [X_train, X_test1, X_test2],
                       [y_train, Y_test1, Y_test2]
                      ):
    print(f'sens: {100*validate.sensitivity_score(y, m.predict(x)):0.1f} \
    spec: {100*validate.specificity_score(y, m.predict(x)):0.1f}')

sens: 95.4     spec: 53.0
sens: 87.7     spec: 52.3
sens: 98.4     spec: 52.3


# break down model

In [100]:
idxs_feats = m.coef_.flatten()!=0
fs = feat_names[idxs_feats]
# vals = m.coef_.flatten()[idxs_feats] > 0
# print(vals)

print(f'{"baseline":>30}\t{100 * y_train.mean():0.2f}%')
print('-------------------------------------')
for i, f1 in enumerate(fs):
    idxs = X_train[f1] == 1
    print(f'{f1:>30}\t{100 * y_train[idxs].mean():0.2f}% ({idxs.sum()})')
print('-------------------------------------')

for i, f1 in enumerate(fs):
    for j, f2 in enumerate(fs[:i]):
        
        idxs = X_train[f1] == 1
        idxs = idxs & (X_train[f2] == 1)
        print(f'{f1 + " " + f2:>48}\t{100 * y_train[idxs].mean():0.2f}% ({idxs.sum()})')

                      baseline	1.62%
-------------------------------------
          AbdTenderDegree_None	0.93% (5616)
 AbdTrauma_or_SeatBeltSign_yes	6.05% (1289)
             GCSScore_Full_yes	1.01% (6935)
-------------------------------------
AbdTrauma_or_SeatBeltSign_yes AbdTenderDegree_None	4.07% (614)
          GCSScore_Full_yes AbdTenderDegree_None	0.24% (4669)
 GCSScore_Full_yes AbdTrauma_or_SeatBeltSign_yes	4.04% (1114)


**lok at errors**

In [102]:
fn = m.predict(X_train) < y_train

In [145]:
df[idxs_train][fn]['Age']

594      2.0
2704     0.0
5977    16.0
9436    17.0
9626     9.0
9971    17.0
Name: Age, dtype: float64